In [ ]:
!pip install openpyxl
!pip install excel
!pip install pandas

import pandas as pd
import excel
import openpyxl

cc = pd.read_excel('INT_DELNT_CRTL_META_REV.xlsx','INT_DELNT_CRTL_META_REV')
RAI = pd.read_excel('rai.xlsx','RAIs')

df = cc[['CC', 'NCR_RAI', 'STATUS']]
df.rename(columns={'NCR_RAI': 'RAI'}, inplace=True)

df = df.fillna('-')

df['STATUS_LIBERA'] = '-'

lista = []
for a in range(len(df)):
  x = df['RAI'][a]
  y = x.split('/')
  if len(y) > 1:
      for b in y:
          lista.append([df['CC'][a], b, df['STATUS'][a], df['STATUS_LIBERA'][a]])
  else:
      lista.append([df['CC'][a], x, df['STATUS'][a], df['STATUS_LIBERA'][a]])

new_df = pd.DataFrame(data=lista, columns=['CC', 'NCR_RAI', 'STATUS', 'STATUS_LIBERA'])
new_df.sort_values(by=['NCR_RAI'], inplace=True)

for i in range(len(new_df)):
  a = new_df['NCR_RAI'].loc[i]
  if a != '-':
      b = new_df[new_df['NCR_RAI'] == new_df['NCR_RAI'].loc[i]]
      e = []
      for c in b['STATUS']:
          if c == 'Publicado':
              e.append(c)
          if len(e) == len(b['STATUS']):
              new_df['STATUS_LIBERA'].loc[i] = 'Todos Cadernos Publicados'

new_df.sort_values(by=['NCR_RAI'], inplace=True)
df2 = RAI[['RAI', 'DISCIPLINA', 'COS', 'STATUS', 'ACAO_DELINEAMENTO']]

df2 = df2[df2['STATUS'] == 'Em Delineamento | Analisado']

analize_rai = new_df[new_df['STATUS_LIBERA'] == 'Todos Cadernos Publicados'][['NCR_RAI', 'STATUS_LIBERA']]
analize_rai = analize_rai[analize_rai['NCR_RAI'] != '']

cont = 0
for a in analize_rai.index:
  cont += 1
  analize_rai.rename(index={a: cont}, inplace=True)

lista = []
cont = 0
for a in analize_rai['NCR_RAI']:
  for b in df2['RAI']:
      if a == b:
          cont += 1
          lista.append([b, 'Liberar RAI_NCR'])

NCR_RAI_LIBERAR = pd.DataFrame(data=lista, columns=['NCR_RAI', 'STATUS'])

LIBERAR = NCR_RAI_LIBERAR[NCR_RAI_LIBERAR['NCR_RAI'] != '']
NCR_RAI_Libera = LIBERAR.groupby(['NCR_RAI', 'STATUS']).count()

NCR_RAI_Libera.to_excel('NCR_RAI_LIBERAR.xlsx', 'NCR_RAI_LIBERAR')

NCR_RAI_Libera